In [7]:
import torch
from sqrll.sqrllm import SqrLLM
import struct

model_file = '../../sqrll/example/models/model2048wu.pt'

model = SqrLLM.load(model_file).eval()

tokenizer = None

params = sum(p.numel() for p in model.parameters())
print(f'{params=:,}')

params=11,698,432


In [8]:
# from tokenizers import Tokenizer
# token_file = '../../sqrll/example/models/tokenizer2048.json'
# tokenizer = Tokenizer.from_file(token_file)
# n_vocab = tokenizer.get_vocab_size()

from btok import Tokenizer
token_file = '../../sqrll/example/models/bpe2048wu.pack'
with open(token_file, 'rb') as f:
    tokenizer = Tokenizer(f.read())
n_vocab = tokenizer.num_tokens()
tokens = [tokenizer.token(i) for i in range(n_vocab)]

print(f'{n_vocab=}')

n_vocab=2048


In [9]:
from torch2cpp import codegen
from sqrll import sqrll, sqrllm

inputs = torch.tensor([[0]])
_, mem = model(inputs)
mem = [torch.zeros_like(m) for m in mem]

with open('build/model.cpp', 'w') as out_file:
    codegen(
        model,
        out_file,
        args=[inputs, mem],
        tokens=tokens,
        autowrap_functions=[
            sqrll.sqrll_kernel,
            sqrllm.rms_norm,
        ],
    )

In [10]:
%%bash
make model.js

em++ build/model.cpp -o build/model.js -I/home/k/dev/torch2cpp/src/torch2cpp/include \
	-Os -s MODULARIZE=1 -s EXPORT_NAME=load_model \
	-s EXPORTED_FUNCTIONS=_model_step,_model_reset,_model_encode,_model_decode,_malloc
